In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import random
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from random import shuffle
import time
from sklearn.manifold import MDS

In [20]:
pip install git+https://github.com/samueljackson92/coranking.git

  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-wkn_x8nd
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-wkn_x8nd
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87031 sha256=60784df97d610b1f263b5bd745f8cb59177d7ad5be536918b5e0833afb1a2fe3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ig0gpxh4/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


In [4]:
link1='https://drive.google.com/open?id=1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
train_df  = pd.read_csv('train.csv')

1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW


In [0]:
concat_df =train_df.copy()

#columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice']
columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice','MasVnrArea', 'GarageYrBlt', 'LotFrontage']
concat_df.drop(columns_to_drop, axis=1, inplace=True)

nan_columns=concat_df.columns[concat_df.isna().any()].tolist()
numerical_cols = [f for f in concat_df.columns if concat_df.dtypes[f] != 'object']
containNumNans = list(set(nan_columns) & set(numerical_cols))

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(concat_df[numerical_cols])
concat_df.loc[:,numerical_cols] = scaled_values

concat_df = pd.get_dummies(concat_df)

In [8]:
labels = train_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(concat_df, labels, test_size=0.25, random_state=42)

GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
GB_model = GBest.fit(X_train, y_train)
preds = GBest.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


RMSE: 24665.666737


In [15]:
concat_df.shape

(1460, 266)

###MDS

In [0]:
from prettytable import PrettyTable
t_mds_10k = PrettyTable(['RMSE_onMDSData', 'Dimensions', 'MDS_time','GBR_time' ,'total_time', 'trust_mds','cont_mds' ,'lcmc_mds'])


def MDS_Decision_Tree (training_data, components,size):
  #Reducing to mentioned components
  #print("starting mds")
  mds_start_time = time.time()
  embedding = MDS(n_components=components)
  training_data_mds = embedding.fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  #print("ending mds")
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()

  GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
  GB_model = GBest.fit(X_train, y_train)
  preds = GBest.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))


  clf_time = time.time() - clf_start_time
  score=rmse
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))


  trust_mds = 0
  cont_mds = 0
  lcmc_mds = 0
  
  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))
  
  #print(components)
  print("{}, {}, {}, {}, {}, {}, {}, {} ".format( score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds))
  print("*"*40)
  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [23]:
dimensions=266
size=1460
while dimensions >= 2:
  MDS_Decision_Tree (concat_df, dimensions,size)
  dimensions=int(dimensions/2)

39629.8339, 266, 136.7145, 41.6658, 178.3802, 0.977, 0.9746, 0.6883 
****************************************
40009.6348, 133, 107.1595, 28.4714, 135.631, 0.9757, 0.9733, 0.6631 
****************************************
41935.2256, 66, 90.2599, 21.9614, 112.2213, 0.9724, 0.9704, 0.6197 
****************************************
40238.4224, 33, 79.9176, 14.8866, 94.8042, 0.9653, 0.9651, 0.5588 
****************************************
44288.7714, 16, 72.5007, 12.3558, 84.8565, 0.9468, 0.9538, 0.4594 
****************************************
44657.3524, 8, 69.7165, 7.9249, 77.6414, 0.9155, 0.9298, 0.3554 
****************************************
48884.2436, 4, 68.6434, 7.389, 76.0323, 0.8697, 0.8963, 0.2421 
****************************************
53504.1514, 2, 68.9442, 4.8575, 73.8017, 0.7762, 0.8344, 0.127 
****************************************


In [24]:
print(t_mds_10k)

+----------------+------------+----------+----------+------------+-----------+----------+----------+
| RMSE_onMDSData | Dimensions | MDS_time | GBR_time | total_time | trust_mds | cont_mds | lcmc_mds |
+----------------+------------+----------+----------+------------+-----------+----------+----------+
|   39629.8339   |    266     | 136.7145 | 41.6658  |  178.3802  |   0.977   |  0.9746  |  0.6883  |
|   40009.6348   |    133     | 107.1595 | 28.4714  |  135.631   |   0.9757  |  0.9733  |  0.6631  |
|   41935.2256   |     66     | 90.2599  | 21.9614  |  112.2213  |   0.9724  |  0.9704  |  0.6197  |
|   40238.4224   |     33     | 79.9176  | 14.8866  |  94.8042   |   0.9653  |  0.9651  |  0.5588  |
|   44288.7714   |     16     | 72.5007  | 12.3558  |  84.8565   |   0.9468  |  0.9538  |  0.4594  |
|   44657.3524   |     8      | 69.7165  |  7.9249  |  77.6414   |   0.9155  |  0.9298  |  0.3554  |
|   48884.2436   |     4      | 68.6434  |  7.389   |  76.0323   |   0.8697  |  0.8963  |  